In [3]:
import os
os.chdir('../')
print(os.getcwd())

/DATA/rahul_1911mt11/multi-tasking/EEG-Multi-Tasking


In [4]:
import torch 
import random
import traceback
import torchvision
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms

In [12]:
# Device configuration
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

# Hyper parameters
episodes = 100
num_epochs = 5
num_classes = 2
batch_size = 20
learning_rate = 0.00001

In [13]:
from src.dataset import multi_task_dataset

In [14]:
df = pd.read_csv('input/data.csv')

In [15]:
df = df.sample(frac=1).reset_index(drop=True)
df.shape

(3480, 5)

In [16]:
def conv_block(in_channels, out_channels):
    '''Convolution Block of 3x3 kernels + batch norm + maxpool of 2x2'''

    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 5, padding=2),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
def set_grad(model, grad):
    for param in model.parameters():
#         print(param.shape)
        param.requires_grad = grad

In [17]:
class multi_task_model(nn.Module):
    def __init__(self):
        super(multi_task_model, self).__init__()

        self.conv1 = conv_block(1, 20)
        self.conv2 = conv_block(20, 40)
        self.conv3 = conv_block(40, 60)        
        self.fc =  nn.Linear(9 * 4 * 60, 100 )    
        
        #heads
        self.y1o = nn.Linear(100,2)
        self.y2o = nn.Linear(100,2)
        self.y3o = nn.Linear(100,2)
        self.y4o = nn.Linear(100, 2)
        self.y5o = nn.Linear(100, 2)
        self.y6o = nn.Linear(100,2)
        self.y7o = nn.Linear(100, 2)
        self.y8o = nn.Linear(100, 2)
        self.y9o = nn.Linear(100, 2)
        
    def freeze(self, subject):
        set_grad(self.y1o, False)
        set_grad(self.y2o, False)
        set_grad(self.y3o, False)
        set_grad(self.y4o, False)
        set_grad(self.y5o, False)
        set_grad(self.y6o, False)
        set_grad(self.y7o, False)
        set_grad(self.y8o, False)
        set_grad(self.y9o, False)
        
        if subject==1:
            set_grad(self.y1o, True)
        if subject==2:
            set_grad(self.y2o, True)
        if subject==3:
            set_grad(self.y3o, True)
        if subject==4:
            set_grad(self.y4o, True)
        if subject==5:
            set_grad(self.y5o, True)
        if subject==6:
            set_grad(self.y6o, True)
        if subject==7:
            set_grad(self.y7o, True)
        if subject==8:
            set_grad(self.y8o, True)
        if subject==9:
            set_grad(self.y9o, True)
            

    def forward(self, x): 
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
#         print('head shape ',x.shape)
        x = x.reshape(x.size(0), -1)
        x1 = self.fc(x)
        
        # heads
        out1 = self.y1o(x1)
        out2 = self.y2o(x1)
        out3 = self.y3o(x1)
        out4 = self.y4o(x1)
        out5 = self.y5o(x1)
        out6 = self.y6o(x1)
        out7 = self.y7o(x1)
        out8 = self.y8o(x1)
        out9 = self.y9o(x1)
                
        #y5o = torch.sigmoid(self.y5o(x1)) #should be sigmoid
#         print('Losses = ',y4o, '\n')
        return out1, out2, out3, out4, out5, out6, out7, out8, out9

In [18]:
# Train the model

for episode in range(episodes):
    n = random.randint(1,9)
    df_train = df.loc[(df.Type=='train') & (df.Subject==n)].reset_index(drop = True)
    train_dataset = multi_task_dataset(df_train)

    # Data loader
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                               batch_size=batch_size, 
                                               shuffle=True)
    try:
        checkpoint = torch.load('model.pth', map_location=torch.device('cuda:1' if torch.cuda.is_available() else 'cpu'))
        model = multi_task_model().to(device)
        model.load_state_dict(checkpoint['weights'])
        model = model.to(device)
#         print('-------Trained model loaded-----------')
    except :
        traceback.print_exc()
        model = multi_task_model()
        model = model.to(device)
        print('-------New model created --------------')



    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    total_step = len(train_loader)
    for epoch in range(num_epochs):
        for i, (images, labels, subjects) in enumerate(train_loader):
            images = images.to(device)
            labels = labels.to(device)
            subjects = subjects.to(device)
            outputs = model(images)
            loss = criterion(outputs[subjects[0].item()-1], labels)

    #         model.freeze(subjects)
    #         model.to(device)
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            torch.save({'weights': model.state_dict()},'model.pth')
            if (i+1) % 5000 == 0:
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                       .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
    print(f"-------------  EPISODE : {episode}  ----------------------")
    print_training_accuracy()
    print_testing_accuracy()

-------------  EPISODE : 0  ----------------------
Train Accuracy : 71.6304347826087 %
Test Accuracy : 72.63888888888889 %
-------------  EPISODE : 1  ----------------------
Train Accuracy : 74.1304347826087 %
Test Accuracy : 71.38888888888889 %
-------------  EPISODE : 2  ----------------------
Train Accuracy : 74.45652173913044 %
Test Accuracy : 72.77777777777777 %
-------------  EPISODE : 3  ----------------------
Train Accuracy : 74.56521739130434 %
Test Accuracy : 73.19444444444444 %
-------------  EPISODE : 4  ----------------------
Train Accuracy : 74.71014492753623 %
Test Accuracy : 73.61111111111111 %
-------------  EPISODE : 5  ----------------------
Train Accuracy : 73.76811594202898 %
Test Accuracy : 73.88888888888889 %
-------------  EPISODE : 6  ----------------------
Train Accuracy : 74.60144927536231 %
Test Accuracy : 73.19444444444444 %
-------------  EPISODE : 7  ----------------------
Train Accuracy : 75.21739130434783 %
Test Accuracy : 73.61111111111111 %
----------

In [1]:
# Test the trained model on train data
def print_training_accuracy():
    df_train = df.loc[(df.Type=='train') & (df.Subject!=10)].reset_index(drop = True)
    train_dataset = multi_task_dataset(df_train)
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                               batch_size=1, 
                                               shuffle=True)
    model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
    with torch.no_grad():
        correct = 0
        total = 0
        for image, label, subject in train_loader:
            image = image.to(device)
            label = label.to(device)
            outputs = model(image)
            _, predicted = torch.max(outputs[subject-1].data, 1)
    #         print('predicted and label = ', predicted.item(), label.item())     

            total += label.size(0)
            correct += (predicted == label).sum().item()
    #         print(predicted, label.item())
    # print(total, correct)
    print('Train Accuracy : {} %'.format(100 * correct / total))

In [2]:
# Test the model on test data
def print_testing_accuracy():
    df_test = df.loc[(df.Type=='test') & (df.Subject!=10)].reset_index(drop = True)
    test_dataset = multi_task_dataset(df_test)
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                               batch_size=1, 
                                               shuffle=True)
    model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
    with torch.no_grad():
        correct = 0
        total = 0
        for image, label, subject in test_loader:
            image = image.to(device)
            label = label.to(device)
            outputs = model(image)
            _, predicted = torch.max(outputs[subject-1].data, 1)
    #         print('predicted and label = ', predicted.item(), label.item())     

            total += label.size(0)
            correct += (predicted == label).sum().item()
    #         print(predicted, label.item())
    # print(total, correct)
    print('Test Accuracy : {} %'.format(100 * correct / total))



In [13]:
import subprocess

def get_gpu_memory_map():
    """Get the current gpu usage.

    Returns
    -------
    usage: dict
        Keys are device ids as integers.
        Values are memory usage as integers in MB.
    """
    result = subprocess.check_output(
        [
            'nvidia-smi', '--query-gpu=memory.used',
            '--format=csv,nounits,noheader'
        ], encoding='utf-8')
    # Convert lines into a dictionary
    gpu_memory = [int(x) for x in result.strip().split('\n')]
    gpu_memory_map = dict(zip(range(len(gpu_memory)), gpu_memory))
    sorted_gpu_memory_map = dict(sorted(gpu_memory_map.items(), key=lambda item: item[1]))
    return sorted_gpu_memory_map

In [14]:
stats = get_gpu_memory_map()

In [17]:
stats

{3: 2153, 4: 3749, 1: 7294, 2: 10258, 0: 10585}

In [24]:
list(stats.keys())[0]

3